# Genetic Algorithm Example

This is a simple illustration of a Genetic Algorithm finding the minimum value for a Travelling Salesperson Problem, for the course Natural Computing course at University of Edinburgh. This is by no means perfect code and should not be taken as such. This code is created by Billy Lyons, and takes some inspiration from https://github.com/ezstoltz/genetic-algorithm, with some changes for ease.

#  Importing necessary libraries

In [1]:
import numpy as np
import random
import operator

# Genes and Chromosomes

In this formulation the individual genes of our solutions consist of cities the agent visits along the route, the chromosome being the full route. First we make a class for each city, that contains their personal coordinates in plane, and a function for computing the distance between a given city and another.

In [3]:
class City:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def distance_to(self, city):
        x = np.abs(self.x - city.x)
        y = np.abs(self.y - city.y)
        distance = np.sqrt((x**2)+(y**2))
        return distance
    
    def __repr__(self):
        return "(" + str(self.x)+","+str(self.y)+")"

Testing gene generation

In [4]:
test_city = City(1,1)
other_city = City(3,5)

print(test_city.distance_to(other_city))
print(test_city)

4.47213595499958
(1,1)


Given that we are able to generate individual cities, we will now create our complete list of cities. This list will be where we generate our population from.

In [5]:
cities = []

for i in range(50):
    cities.append(City((np.random.randint(0, 50)), np.random.randint(0,50)))
    
print(cities)

[(33,39), (48,15), (12,10), (6,42), (43,22), (1,22), (10,23), (38,10), (27,34), (48,20), (33,26), (40,14), (42,41), (27,22), (30,46), (11,12), (17,21), (22,7), (5,18), (26,33), (32,26), (19,34), (2,3), (10,17), (10,11), (24,37), (22,30), (8,20), (3,0), (8,42), (45,19), (34,34), (3,8), (3,5), (5,16), (9,24), (15,18), (0,25), (25,22), (13,12), (12,0), (13,18), (26,4), (17,15), (2,10), (0,25), (42,14), (25,30), (48,5), (28,15)]


Now that we have all of our cities generated and added into a list. We will define the fuction which generates a route, this route will be a member of our population, and from this we will create our initial population. Each member of the population must vist every state in city, and return to the start, so these are sampled without replacement.

In [6]:
def create_route(cities):
    route = random.sample(cities, len(cities))
    return route

Create initial population

In [7]:
def initial_population(pop, cities):
    population = []
    
    for i in range(pop):
        population.append(create_route(cities))
    return population

# Fitness

In a genetic algorithm we need to know how good our individuals are for a number of reasons. If we are performing roulette wheel selection, we weight likelihood to breed and thus pass on genes by fitness, if we have some percentage of elitism, we may wish to keep the X most successful members of the population. As such we must define a fitness function for our routes.

In [8]:
def fitness(route):
    distance, fitness = 0.0,0.0
    for i in range(0, len(route)):
        from_city=route[i]
        to_city=route[(i+1)%len(route)]
        
        distance+=(from_city.distance_to(to_city))
    
    fitness=(1/distance)
    
    return fitness, distance

# Genetic Algorithm

We now have a population and a way to calculate fitness. We must use these to rank our population from best to worst. This is essential, as, you should weight the chance to reproduce by the fitness of each member, much in the same way that a better biological agent is more likely to pass on their genes with greater succes. Additionally, you may wish to maintain some elite members through to the next generation, so ordering is natural and makes life a lot easier.

In [10]:
def ranking_pop(population):
    fitnesses = {}
    sum_fit = 0.0
    sum_dist = 0.0
    distances = {}
    for i in range(len(population)):
        fitnesses[i], distances[i] = fitness(population[i])
        x, y = fitness(population[i])
        sum_fit += x
    return sorted(fitnesses.items(), key=operator.itemgetter(1), reverse=True), sum_fit, distances

Now that we have our sorted list, we must perform selection for the next parents of the next generation. Here we are going to replicate into the next generation a certain number of elite agents, and we take the population size minus this elitism, and draw that many members from the generation by a probability distribution which is determined by their individual fitness.

1) Why might we want to have some level of elitism?

2) Why do we use a weighted selection?

In [11]:
def selection(ranked, tot, elitism):
    probabilities = np.zeros(len(ranked))
    members = np.arange(len(ranked))
    size = len(ranked)
    elite_members = []
    for i in range(len(ranked)):
        x,y = ranked[i][0], ranked[i][1]
        if i < elitism:
            elite_members.append(x)
        probabilities[x] = y
    probabilities=probabilities/np.sum(probabilities)
    selected_members = np.random.choice(members, size-elitism, p=probabilities)
    return selected_members, elite_members

In [12]:
def mating_pool(population, selected_members, elite_members):
    mating_pool = []
    elite = []
    for i in range(len(selected_members)):
        index = selected_members[i]
        mating_pool.append(population[index])
                
    for i in range(len(elite_members)):
        index = elite_members[i]
        elite.append(population[index])        
    return mating_pool, elite

Now that we have our pool of viable mates, we randomly draw some beginning and end point for the crossover between the two chromosomes. Maintaining order.

1) Look at the below, what is happening? How is this different from point crossover in the all ones problem and the class notes?

2) Why is it important to maintaining ordering?

3) Crossover here occurs every time. How might you change this to a probabilistic method? Why might that be better/worse?


In [13]:
def breed_from_parents(parent1, parent2):
    child = []
    childP1 = []
    childP2 = []
    t1,t2 = 0,0
    geneA = int(np.random.rand()*len(parent1))
    geneB = int(np.random.rand()*len(parent1))
    while geneB == geneA:
        geneB = int(np.random.rand()*len(parent1))
    
    start = min(geneA, geneB)
    end = max(geneA, geneB)
    
    for i in range(start, end):
        childP1.append(parent1[i])
    childP2 = [item for item in parent2 if item not in childP1]
    
    for i in range(len(parent1)):
        if i>=start and i<end:
            child.append(childP1[t1])
            t1+=1
        else:
            child.append(childP2[t2])
            t2+=1
    return child

In [14]:
def new_population(mating_pool, elite_pool):
    children = []
    pool = random.sample(mating_pool, len(mating_pool))
        
    for i in range(len(elite_pool)):
        children.append(elite_pool[i])
    for i in range(len(mating_pool)):
        child = breed_from_parents(pool[i], pool[len(mating_pool)-i-1])
        children.append(child)
    return children

Now that our parent population has breeded and we have generated our new population from the children and the elites who have lived into the next generation, we must go through each solution and check to see if any must be mutated. We are doing this by running through each candidate in the new population, and at each point in the chromosome we see if it will mutate here and then perform a swap.

1) Why is mutation an important part of a GA (hint: Think about the search space)?

2) If you had a chromosome reprsenting some real number e.g. 3.14159 -> chromosome 314159, how might you adapt the mutation rate?


In [15]:
def mutate(chromosome, prob_mut):
    for mutable in range(len(chromosome)):
        if (np.random.rand()<prob_mut):
            swap = np.random.randint(0, len(chromosome))
            
            cityA = chromosome[mutable]
            cityB = chromosome[swap]
            
            chromosome[mutable]=cityB
            chromosome[swap]=cityA
    return chromosome

In [16]:
def mutation_over_pop(population, prob_mut):
    mutated_pop = []
    for i in range(len(population)):
        mutated_chromo = mutate(population[i], prob_mut)
        mutated_pop.append(mutated_chromo)
    return mutated_pop

In [18]:
def new_generation(current_gen, elitism, prob_mut):
    rank, tot, distances = ranking_pop(current_gen)
    selected_members, elite_members = selection(rank, tot, elitism)
    mates, elites = mating_pool(current_gen, selected_members, elite_members)
    children = new_population(mates, elites)
    next_gen = mutation_over_pop(children, prob_mut)
    return next_gen    

Now we put everything together and run it for the generations!

In [19]:
def genetic_algorithm(population, elitism, prob_mut, generations):
    pop = initial_population(population, cities)
    ranked, tot, distances = ranking_pop(pop)
    print("Initial shortest distance: {}".format(distances[ranked[0][0]]))
    
    for i in range(generations):
        pop = new_generation(pop, elitism, prob_mut)
    
    ranked, tot, distances = ranking_pop(pop)
    print("Final shortest distance: {}".format(distances[ranked[0][0]]))
    best_route = pop[ranked[0][0]]
    return best_route

In [20]:
genetic_algorithm(100, 3, 0.01, 50)

Initial shortest distance: 1045.1388769296495
Final shortest distance: 885.7632167879109


[(30,46),
 (24,37),
 (28,15),
 (26,4),
 (10,11),
 (13,18),
 (8,42),
 (9,24),
 (0,25),
 (33,39),
 (19,34),
 (25,30),
 (34,34),
 (40,14),
 (27,22),
 (2,3),
 (26,33),
 (45,19),
 (48,15),
 (48,20),
 (42,14),
 (3,0),
 (13,12),
 (8,20),
 (3,8),
 (27,34),
 (38,10),
 (2,10),
 (22,7),
 (12,0),
 (32,26),
 (33,26),
 (48,5),
 (43,22),
 (42,41),
 (0,25),
 (1,22),
 (10,17),
 (12,10),
 (5,16),
 (6,42),
 (15,18),
 (17,15),
 (25,22),
 (17,21),
 (22,30),
 (10,23),
 (5,18),
 (3,5),
 (11,12)]

# Suggestions for additional exercises

1) Add code to plot graphs, plot the cities and the route taken by the best solution after 50 generations.

2) Create your own GA, it could be as simple as the all ones problem, having your GA learn to output a string that you enter, or something more exciting!

3) This is very much the canonical GA (though the crossover is slightly different due to the problem scenario), find a paper that shows some addition or alteration to the canonical algorithm, how is it different, and what is the impact?
